# Demo

In [1]:
import numpy as np
from activations import *
from layers import *
from losses import *
from optimizers import *
from model import *

## ANN Example

In [8]:
p, obs, hidden, out = 4, 5, 3, 2
np.random.seed(1)
x = np.random.rand(p, obs)
oneHot = np.random.multinomial(1,[1/out for i in range(out)], obs).T
labels = np.argmax(oneHot, axis=0)
#labels = np.random.binomial(1, 0.75, obs).reshape(1,30)

model = neuralNet(softmaxWithXentropy, sgd(learnRate=0.1))
model.add(dense((hidden, p), sigmoid))
model.add(dropout((hidden, hidden)))
model.add(dense((out, hidden), identity))
model.fit(x, labels, epochs=31)

Epoch: 0; Cost: 0.9469371833814882
Epoch: 10; Cost: 0.6523658858592422
Epoch: 20; Cost: 0.568316804654173
Epoch: 30; Cost: 0.42484703848843575


In [10]:
p, obs, hidden, out = 4, 5, 3, 2
np.random.seed(1)
x = np.random.rand(p, obs)
oneHot = np.random.multinomial(1,[1/out for i in range(out)], obs).T
labels = np.argmax(oneHot, axis=0)
#labels = np.random.binomial(1, 0.75, obs).reshape(1,30)

model = neuralNet(xentropy, sgd(learnRate=0.1))
model.add(dense((hidden, p), sigmoid))
model.add(dropout((hidden, hidden)))
model.add(dense((out, hidden), softmax))
model.fit(x, labels, epochs=31)

Epoch: 0; Cost: 0.9469371833814882
Epoch: 10; Cost: 0.6523658858592422
Epoch: 20; Cost: 0.568316804654173
Epoch: 30; Cost: 0.42484703848843575


## CNN Example

In [4]:
obs, H, W, channels, out = 10, 28, 28, 3, 3
np.random.seed(1)
x = np.random.randn(obs, H, W, channels)
oneHot = np.random.multinomial(1,[1/out for i in range(out)], obs).T
labels = np.argmax(oneHot, axis=0)

model = neuralNet(softmaxWithXentropy, sgd(learnRate=0.1, momentum=0.9))
model.add(conv2D((28,28,3), 4, 3, 1, 1, 'channels_last'))
model.add(batchNorm(((28,28,4),(28,28,4))))
model.add(activation(((28,28,4),(28,28,4)), relu))
model.add(pooling2D((28,28,4), 2, 2, mode='max'))
model.add(conv2D((14,14,4), 8, 3, 1, 1, 'channels_last'))
model.add(batchNorm(((14,14,8),(14,14,8))))
model.add(activation(((14,14,8),(14,14,8)), relu))
model.add(pooling2D((14,14,8), 2, 2, mode='max'))
model.add(conv2D((7,7,8), 3, 7, 1, 0, 'channels_last'))
model.add(batchNorm(((1,1,3),(1,1,3))))
model.add(flatten((3,(1,1,3))))
model.fit(x, labels, epochs=11)

Epoch: 0; Cost: 1.3100256957707992
Epoch: 10; Cost: 1.0941671311963364


## RNN Example (M:M)

In [4]:
np.random.seed(1)
p, hidden1, hidden2, obs, steps, out = 3, 5, 7, 10, 4, 3
X = np.random.randn(p,obs,steps)
oneHot = [np.random.multinomial(1,[1/out for i in range(out)], obs).T for t in range(steps)]
oneHot = np.stack(oneHot, axis=2)
labels = [np.argmax(oneHot[:,:,t], axis=0) for t in range(steps)]
labels = np.stack(labels, axis=1)

model = neuralNet(softmaxWithXentropy, rmsProp(learnRate=0.01), recurrentLoss=True)
model.add(lstm((hidden1,p), steps, returnSeq=True))
model.add(lstm((hidden2,hidden1), steps, returnSeq=True))
model.add(denseThruTime((out,hidden2), steps=steps, activation=identity))
model.fit(X, labels, epochs=31)

Epoch: 0; Cost: 4.41306454641426
Epoch: 10; Cost: 4.065282350473246
Epoch: 20; Cost: 3.48286550957244
Epoch: 30; Cost: 2.7659792652782844


## RNN Example (M:1)

In [5]:
np.random.seed(1)
p, hidden1, hidden2, obs, steps, out = 3, 5, 7, 10, 4, 3
X = np.random.randn(p,obs,steps)
oneHot = np.random.multinomial(1,[1/out for i in range(out)], obs).T
labels = np.argmax(oneHot, axis=0)

model = neuralNet(softmaxWithXentropy, rmsProp(learnRate=0.01))
model.add(lstm((hidden1,p), steps, returnSeq=True))
model.add(lstm((hidden2,hidden1), steps))
model.add(dense((out, hidden2), identity))
model.fit(X, labels, epochs=31)

Epoch: 0; Cost: 1.0827380944742642
Epoch: 10; Cost: 0.6417763307257696
Epoch: 20; Cost: 0.32321272495683184
Epoch: 30; Cost: 0.21462404347650818


## RNN Example (Encoder-Decoder)

In [5]:
np.random.seed(1)
p, h_encode, h_decode, obs, t_encode, t_decode, out = 3, 5, 6, 10, 4, 5, 3
X = np.random.randn(p,obs,t_encode)
oneHot = [np.random.multinomial(1,[1/out for i in range(out)], obs).T for t in range(t_decode)]
oneHot = np.stack(oneHot, axis=2)
labels = [np.argmax(oneHot[:,:,t], axis=0) for t in range(t_decode)]
labels = np.stack(labels, axis=1)

model = neuralNet(softmaxWithXentropy, adam(learnRate=0.1), recurrentLoss=True)
model.add(rnn((h_encode,p), t_encode))
model.add(repeatVector((h_encode, h_encode), t_decode))
model.add(rnn((h_decode,h_encode), t_decode, returnSeq=True))
model.add(denseThruTime((out,h_decode), steps=t_decode, activation=identity))
model.fit(X, labels, epochs=31)

Epoch: 0; Cost: 5.50220057580703
Epoch: 10; Cost: 3.1687579668521932
Epoch: 20; Cost: 1.8497153583184835
Epoch: 30; Cost: 1.1859934982056743
